In [77]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gujarat-biomass-production/Distance_Matrix.csv
/kaggle/input/gujarat-biomass-production/preds1.csv
/kaggle/input/gujarat-biomass-production/sample_submission.csv
/kaggle/input/gujarat-biomass-production/Biomass_History.csv
/kaggle/input/gujarat-biomass-production/preds2.csv


In [78]:
distance=pd.read_csv("/kaggle/input/gujarat-biomass-production/Distance_Matrix.csv",index_col=0)
biomass=pd.read_csv('/kaggle/input/gujarat-biomass-production/Biomass_History.csv')
samplesubmission=pd.read_csv('/kaggle/input/gujarat-biomass-production/sample_submission.csv')

In [79]:
pred1=pd.read_csv('/kaggle/input/gujarat-biomass-production/preds1.csv',index_col=0)

In [18]:
pred1.sum()

2018    283612.983246
2019    327111.628956
dtype: float64

In [19]:
def truncate(f, n):
    '''Truncates/pads a float f to n decimal places without rounding'''
    s = '%.12f' % f
    i, p, d = s.partition('.')
    return float('.'.join([i, (d+'0'*n)[:n]]))

In [20]:
import plotly.express as px

color=pred1['2018']
fig = px.scatter_mapbox(biomass, 
                        lat="Latitude", 
                        lon="Longitude", 
                        hover_name="Index", 
                        hover_data=["Index"],
                        color=color,
                        zoom=5, 
                        height=400,
                        width=600)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

- total biomass 3.84lac

In [80]:
biomass_pred=pred1
# val=(pred1['2018'].values+pred1['2019'].values)/2
val=pred1['2018'].values

In [81]:
dist=distance.values.copy()
wt=val[:,np.newaxis]+val[np.newaxis,:]
for i in range(distance.shape[0]):
    wt[i][i]=0
    dist[i][i]=1
weight=wt/dist
weight[weight==np.inf]=1e9

/tmp/ipykernel_32/252930380.py:6: RuntimeWarning:

divide by zero encountered in divide



In [82]:
%%time
class DSU:
    def __init__(self):
        self.p = {}
        self.count = 0
        self.biomass={}
        self.initvalue={}
        self.partialjoin=[]
    
    def add(self, x,val):
        if x in self.p: return
        self.p[x] = x
        self.count += 1
        self.biomass[x]=val
        self.initvalue[x]=val
    
    def find(self, x):
        if x != self.p[x]:
            self.p[x] = self.find(self.p[x])
        return self.p[x]
    
    def union(self, x, y):
        xp, yp = self.find(x), self.find(y)
        if xp == yp: return False
        self.p[xp] = yp
        self.biomass[yp]+=self.biomass[xp]
        self.count -= 1
        return True
    
    def partialunion(self,x,y,amount):
        cap=amount-self.biomass[y]
        biomassperdist=[]
        for node in range(2418):
            if self.find(node)==x:
                depo=y
                wt=1e9
                if dist[node][depo]!=0: 
                    wt=self.initvalue[node]/dist[node][depo]
                value=self.initvalue[node]
                biomassperdist.append([wt,value,node])
        biomassperdist=sorted(biomassperdist,key=lambda l:l[0], reverse=True)
        for index,node in enumerate(biomassperdist):
            if node[1]<=cap:
                self.biomass[node[2]]=self.initvalue[node[2]]
                self.biomass[y]+=node[1]
                self.p[node[2]]=y
                cap-=node[1]
            elif cap>=0.001:
                self.biomass[node[2]]=self.initvalue[node[2]]
                self.biomass[y]+=cap
                self.biomass[node[2]]-=cap
                self.initvalue[node[2]]-=cap
                biomassperdist[index][1]-=cap
                if cap>0:
                    self.partialjoin.append([cap,node[2],depo])
                cap=0
                break
        mask=[]
        value=[]
        for node in biomassperdist[index:]:
            mask.append(node[2])
            value.append(node[1])
#         print(np.sum(value))
        tempmatrix=distance.iloc[mask,mask].T.values
        tempcost=np.dot(tempmatrix,value)
        tempindex=np.argmin(tempcost)
        parent=mask[tempindex]
        self.p[parent]=parent
        self.biomass[parent]=self.initvalue[parent]
        for index,i in enumerate(mask):
            #problem parent adding its own value to make it 2 times
            if i!=parent:
                self.p[i] = parent
                self.biomass[i]=value[index]
                self.initvalue[i]=value[index]
                self.biomass[parent]+=value[index]
#         print('biomass:',parent,self.biomass[parent])
    
    def is_connected(self, x, y):
        if x in self.p and y in self.p:
            return self.find(x) == self.find(y)
        else:
            return False

CPU times: user 30 µs, sys: 1 µs, total: 31 µs
Wall time: 36 µs


In [83]:
%%time
edges=[]
for i in range(2418):
    for j in range(2418):
        if i!=j:
            edges.append([weight[i][j],i,j])
edges=sorted(edges,key=lambda l:l[0], reverse=True)
dsu=DSU()
for i in range(2418):
    dsu.add(i,val[i])
for index in range(len(edges)):
    edge=edges[index]
    value=edge[0]
    i=edge[1]
    j=edge[2]
    if dsu.biomass[i]<dsu.biomass[j] and dsu.p[i]==i and dsu.p[j]==j and dsu.biomass[j]<20000:
        if dsu.biomass[j]+dsu.biomass[i]<=20000:
            dsu.union(i,j)
        else:
            dsu.partialunion(i,j,20000)

CPU times: user 43.1 s, sys: 1 s, total: 44.1 s
Wall time: 43.6 s


In [28]:
dsu.union(2399,2321)

True

In [84]:
cluster=[]
for i in range(2418):
    cluster.append(dsu.find(i))
total=0
for j in np.unique(cluster):
    value=[]
    for i in range(2418):
        if dsu.find(i)==j:
            value.append(dsu.initvalue[i])
    for i in dsu.partialjoin:
        if dsu.find(i[2])==j:
            value.append(i[0])
    total+=np.sum(value)
    print(j,np.sum(value))
print(total)

75 20000.0
495 20000.000000000004
648 4277.08842255201
687 19762.29097575
929 19999.999999999996
981 19959.556505273802
1020 19824.533431877804
1291 19977.566085244
1408 19928.350732387793
1518 19904.031159211998
1607 19999.999999999996
1753 19999.999999999996
1796 19998.624114216793
1936 19981.120380081804
2213 19999.8214396892
283612.98324628524


In [ ]:
'''%%time
class DSU:
    def __init__(self):
        self.p = {}
        self.count = 0
        self.biomass={}
        self.partialjoin=[]
    
    def add(self, x,val):
        if x in self.p: return
        self.p[x] = x
        self.count += 1
        self.biomass[x]=val
    
    def find(self, x):
        if x != self.p[x]:
            self.p[x] = self.find(self.p[x])
        return self.p[x]
    
    def union(self, x, y):
        xp, yp = self.find(x), self.find(y)
        if xp == yp: return False
        self.p[xp] = yp
        self.biomass[yp]+=self.biomass[xp]
        self.count -= 1
        return True
    
    def partialunion(self,x,y,amount):
        xp, yp = self.find(x), self.find(y)
        val=amount-self.biomass[yp]
        self.biomass[yp]+=val
        self.biomass[xp]-=val
        self.partialjoin.append([val,xp,yp])
        return val
    
    def is_connected(self, x, y):
        if x in self.p and y in self.p:
            return self.find(x) == self.find(y)
        else:
            return False
        
        
        
        
edges=[]
for i in range(2418):
    for j in range(2418):
        if i!=j:
            edges.append([weight[i][j],i,j])
edges=sorted(edges,key=lambda l:l[0], reverse=True)
dsu=DSU()
for i in range(2418):
    dsu.add(i,val[i])
for index in range(len(edges)):
    edge=edges[index]
    value=edge[0]
    i=edge[1]
    j=edge[2]
    if dsu.biomass[i]<dsu.biomass[j] and dsu.p[i]==i and dsu.p[j]==j and dsu.biomass[j]<20000:
        if dsu.biomass[j]+dsu.biomass[i]<=20000:
            dsu.union(i,j)
        else:
            reduce=dsu.partialunion(i,j,20000)
            for index2 in range(index+1,len(edges)):
                edge2=edges[index2]
                if(edge2[1]==i or edge2[2]==i) and dist[edge2[1]][edge2[2]]!=0 :
                    edges[index][0]-=(reduce/dist[edge2[1]][edge2[2]])
            edges=edges[:index+1]+sorted(edges[index+1:],key=lambda l:l[0], reverse=True)'''

In [85]:
dsu.partialjoin

[[97.4650786960006, 1812, 1753],
 [24.975155692000868, 819, 495],
 [12.485364013999117, 950, 1291],
 [45.54261440979823, 546, 929],
 [21.492413863998422, 1496, 1607],
 [50.84897783779735, 1980, 2213],
 [157.9918055952003, 1526, 1518],
 [152.806964703999, 310, 75],
 [117.65402462419647, 550, 1408],
 [52.33926837159396, 1505, 1796],
 [92.37677366180225, 2355, 1936],
 [1.7027625191948133, 382, 687],
 [81.3298949595889, 489, 981],
 [24.199909096589252, 1765, 1020]]

In [86]:
uniquecluster=np.unique(cluster)
noofcluster=len(uniquecluster)
clustermask=[]
clustermap={}
for i in range(noofcluster):
    clustermap[uniquecluster[i]]=i
    clustermask.append([])
for i in range(2418):
    value=clustermap[cluster[i]]
    clustermask[value].append(i)
    

depoloc=[]
for index,mask in enumerate(clustermask):
    tempmatrix=distance.iloc[mask,mask].T.values
    tempvalue=val[mask]
    tempcost=np.dot(tempmatrix, tempvalue)
    partialedge=[]
    for i in dsu.partialjoin:
        if dsu.find(i[2])==uniquecluster[index]:
            partialedge.append([i[0],i[1]])
    for index,i in enumerate(mask):
        cost=0
        for j in partialedge:
            cost+=(j[0]*dist[j[1]][i])
        tempcost[index]+=cost
    tempindex=np.argmin(tempcost)
    depoloc.append(mask[tempindex])

In [87]:
oldtonewdepomap={}
newtoolddepomap={}
for i in range(noofcluster):
    old=uniquecluster[i]
    new=depoloc[i]
    oldtonewdepomap[old]=new
    newtoolddepomap[new]=old

depomap={}
for index,depo in enumerate(depoloc):
    depomap[depo]=index

In [88]:
biomassperdist=[]
for node in range(2418):
    stats=[]
    for depo in depoloc:
        wt=1e9
        if dist[node][depo]!=0: 
            wt=val[node]/dist[node][depo]
        value=val[node]
        stats.append([wt,value,node,depo])
    stats=sorted(stats,key=lambda l:l[0], reverse=True)
    biomassperdist.append(stats)
biomassperdist=sorted(biomassperdist,key=lambda l:l[0][0], reverse=True)

In [89]:
knapsack={}
capacity={}
for depo in depoloc:
    capacity[depo]=20000
    knapsack[depo]=[0]*2418
index=0
while index<2418:
    stats=biomassperdist[index]
    node=stats[0]
    depo=node[3]
    if capacity[depo]>=node[1]:
        knapsack[depo][node[2]]=node[1]
        capacity[depo]-=node[1]
        index+=1
    else:
        red=capacity[depo]
        knapsack[depo][node[2]]=capacity[depo]
        capacity[depo]=0
        del stats[0]
        for i in range(len(stats)):
            stats[i][1]=(stats[i][1]-red)
            stats[i][0]-=(red/dist[stats[i][2]][stats[i][3]])
        biomassperdist[index]=sorted(stats,key=lambda l:l[0], reverse=True)
        biomassperdist=biomassperdist[:index]+sorted(biomassperdist[index:],key=lambda l:l[0][0], reverse=True)

In [90]:
total=[]
for depo in depoloc:
    total.append(knapsack[depo])
cap=np.sum(total,axis=1)

In [91]:
depovalue=[]
for depo in depoloc:
    mask=knapsack[depo]
    cost=0
    for node,value in enumerate(mask):
        if(value>0):
            value=truncate(value,3)
            cost+=value
    depovalue.append(cost)
depovalue

[19995.73400000002,
 19999.949999999997,
 5884.023999999997,
 18552.734999999997,
 19999.920000000013,
 19999.934,
 19179.929999999986,
 19999.935999999994,
 19999.942000000003,
 19999.938999999995,
 19999.954999999994,
 19999.975000000006,
 19999.942000000006,
 19999.925999999996,
 19999.945999999996]

In [92]:
depoloc

[102,
 346,
 648,
 687,
 929,
 1091,
 882,
 1174,
 1472,
 1456,
 1548,
 1860,
 1849,
 2049,
 2129]

In [ ]:
'''import plotly.express as px

fig = px.scatter_mapbox(biomass.iloc[samplesubmission[samplesubmission['data_type']=='depot_location'].source_index.values], 
                        lat="Latitude", 
                        lon="Longitude", 
                        hover_name="Index", 
                        hover_data=["Index", "2010"],
#                         color='black',
                        zoom=5, 
                        height=400,
                        width=600)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()'''

In [93]:
import plotly.express as px

fig = px.scatter_mapbox(biomass.iloc[depoloc], 
                        lat="Latitude", 
                        lon="Longitude", 
                        hover_name="Index", 
                        hover_data=["Index", "2010"],
                        zoom=5, 
                        height=400,
                        width=600)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [94]:
depoprod={}
for index,depo in enumerate(depoloc):
    depoprod[depo]=depovalue[index]

In [87]:
# hardcode refinery clusters
refmask=[[1037,1324,1687,1907,1746],
        [988,1641,2030,1813,1470],
        [148,865,547,687,442],
        [1283,1291,1488,1937,2102]]

In [40]:
# hardcode refinery clusters for part2 with correct clusters using partialunion 
refmask=[[2321,1718,1432,1232],
        [2172,1860,1905],
        [102,609,443,881],
        [985,1325,1333,1402,1531]]

In [95]:
refmask=[[687,882,1174,1548,2049],
        [1456,1849,1472,1860,2129],
        [648,1091,929,346,102]]

In [96]:
refloc=[]
minicost=0
for index,mask in enumerate(refmask):
    tempmatrix=distance.iloc[mask,:].T.values
    tempvalue=[]
    for i in mask:
        tempvalue.append(depoprod[i])
    tempcost=np.dot(tempmatrix, tempvalue)
    tempindex=np.argmin(tempcost)
    minicost+=np.min(tempcost)
    
    refloc.append(tempindex)
print(minicost)

28209730.2511582


In [97]:
refloc

[1121, 1806, 929]

In [90]:
'''%%time
mini=1e18
refinerylocation=[]
temprefmask=[[],[],[],[]]
refmask=[]
itercount=0
def solve(index,depoloc):
    global mini,refinerylocation,temprefmask,refmask,itercount
    if index>=len(depoloc):
        refloc=[]
        minicost=0
        for index,mask in enumerate(temprefmask):
            tempmatrix=distance.iloc[mask,:].T.values
            tempvalue=[]
            for i in mask:
                tempvalue.append(depoprod[i])
            tempcost=np.dot(tempmatrix, tempvalue)
            tempindex=np.argmin(tempcost)
            minicost+=np.min(tempcost)
            refloc.append(tempindex)
        if(minicost<mini):
            mini=minicost
            refinerylocation=refloc
            refmask=temprefmask
        if(itercount%10000==0):
            print(itercount,mini,refinerylocation)
        itercount+=1
        return
    
    for i in range(4):
        if len(temprefmask[i])<5:
            temprefmask[i].append(depoloc[index])
            solve(index+1,depoloc)
            temprefmask[i].remove(depoloc[index])
    return

solve(0,depoloc)'''

0 49263703.15581255 [547, 988, 1583, 1813]
10000 33867793.11742823 [547, 1148, 1424, 1746]
20000 33867793.11742823 [547, 1148, 1424, 1746]
30000 33867793.11742823 [547, 1148, 1424, 1746]
40000 33867793.11742823 [547, 1148, 1424, 1746]
50000 33867793.11742823 [547, 1148, 1424, 1746]
60000 33867793.11742823 [547, 1148, 1424, 1746]
70000 33867793.11742823 [547, 1148, 1424, 1746]
80000 33867793.11742823 [547, 1148, 1424, 1746]
90000 33867793.11742823 [547, 1148, 1424, 1746]
100000 33867793.11742823 [547, 1148, 1424, 1746]
110000 29809781.11131997 [547, 1359, 1148, 1746]
120000 29809781.11131997 [547, 1359, 1148, 1746]
130000 29809781.11131997 [547, 1359, 1148, 1746]
140000 29809781.11131997 [547, 1359, 1148, 1746]
150000 29809781.11131997 [547, 1359, 1148, 1746]
160000 29809781.11131997 [547, 1359, 1148, 1746]
170000 29809781.11131997 [547, 1359, 1148, 1746]
180000 29809781.11131997 [547, 1359, 1148, 1746]
190000 29809781.11131997 [547, 1359, 1148, 1746]


KeyboardInterrupt: 

In [92]:
refloc=refinerylocation
refloc

[547, 1359, 1148, 1746]

In [206]:
def knapsackmask(val,dist,depoloc):
        biomassperdist=[]
        for node in range(2418):
            stats=[]
            for depo in depoloc:
                wt=1e9
                if dist[node][depo]!=0: 
                    wt=val[node]/dist[node][depo]
                value=val[node]
                stats.append([wt,value,node,depo])
            stats=sorted(stats,key=lambda l:l[0], reverse=True)
            biomassperdist.append(stats)
        biomassperdist=sorted(biomassperdist,key=lambda l:l[0][0], reverse=True)
        knapsack={}
        capacity={}
        for depo in depoloc:
            capacity[depo]=20000
            knapsack[depo]=[0]*2418
        index=0
        while index<2418:
            stats=biomassperdist[index]
            node=stats[0]
            depo=node[3]
            if capacity[depo]>=node[1]:
                knapsack[depo][node[2]]=node[1]
                capacity[depo]-=node[1]
                index+=1
            else:
                totalcap=0
                for tempdepo in depoloc:
                    totalcap+=capacity[tempdepo]
                if totalcap<0.001:
                    break
                red=capacity[depo]
                knapsack[depo][node[2]]=capacity[depo]
                capacity[depo]=0
                del stats[0]
                if(len(stats)==0):
                    index+=1
                    continue
                for i in range(len(stats)):
                    stats[i][1]=(stats[i][1]-red)
                    stats[i][0]-=(red/dist[stats[i][2]][stats[i][3]])
                biomassperdist[index]=sorted(stats,key=lambda l:l[0], reverse=True)
                biomassperdist=biomassperdist[:index]+sorted(biomassperdist[index:],key=lambda l:l[0][0], reverse=True)
        return knapsack

In [207]:
knapsack18=knapsackmask(pred1['2018'].values,dist,depoloc)
knapsack19=knapsackmask(pred1['2019'].values,dist,depoloc)

In [211]:
total=[]
for depo in depoloc:
    total.append(knapsack18[depo])
(np.sum(total,axis=0)<=pred1['2018'].values).sum()

2418

In [212]:
location=[]
for i in depoloc:
    location.append([20182019,'depot_location',i,np.nan,np.nan])
for i in refloc:
    location.append([20182019,'refinery_location',i,np.nan,np.nan])
loc_df=pd.DataFrame(location,columns=samplesubmission.columns)

In [213]:
biomass_forcast=samplesubmission[samplesubmission['data_type']=='biomass_forecast']
pred=np.concatenate((pred1['2018'],pred1['2019']))
biomass_forcast['value']=pred

/tmp/ipykernel_32/3387434358.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [214]:
transport=[]
# depo
depovalue=[[],[]]
#2018
for depo in depoloc:
    mask=knapsack18[depo]
    cost=0
    for node,value in enumerate(mask):
        if(value>0):
#             value=int(value)
            value=truncate(value,1)
            transport.append([2018,'biomass_demand_supply',node,depo,value])
            cost+=value
    depovalue[0].append(cost)
#2019
for depo in depoloc:
    mask=knapsack19[depo]
    cost=0
    for node,value in enumerate(mask):
        if(value>0):
#             value=int(value)
            value=truncate(value,1)
            transport.append([2019,'biomass_demand_supply',node,depo,value])
            cost+=value
    depovalue[1].append(cost)

#ref
#2018
for index,mask in enumerate(refmask):
    for i in mask:
        value=depovalue[0][depomap[i]]
        source=i
        destination=refloc[index]
        transport.append([2018,'pellet_demand_supply',source,destination,value])
#2019
for index,mask in enumerate(refmask):
    for i in mask:
        value=depovalue[1][depomap[i]]
#         print(value)
        source=i
        destination=refloc[index]
        transport.append([2019,'pellet_demand_supply',source,destination,value])
transport_df=pd.DataFrame(transport,columns=samplesubmission.columns)

loc_df+biomass_forcast+transport_df

In [215]:
submission=pd.concat([loc_df,biomass_forcast,transport_df],axis=0)

In [216]:
submission.to_csv('submission_with_pred_partialunion_3refsol.csv',index=False)